In [344]:
import os
import numpy as np
import scipy

from scipy.optimize import curve_fit
from scipy.integrate import simpson
from scipy.optimize import minimize
import scipy.optimize as opt

# Set print options to display the full array
np.set_printoptions(threshold=np.inf)

new_dir_path = "/Users/mohammadmaster/Desktop"
os.chdir(new_dir_path)

from CHN_reader_F import Spectrum

#spectrum_instance = Spectrum('Sample_30_B_338.Chn')
#print(spectrum_instance.channel)
#print(len(spectrum_instance.channel))

directory = "/Users/mohammadmaster/Desktop/2022-12-27_Montserrat_sample_32"

# Get a list of all files in the directory with extension .Chn
file_list = [f for f in os.listdir(directory) if f.endswith('.Chn')]

# Create a list of Spectrum instances
spectrum_list = []
for file_name in file_list:
    file_path = os.path.join(directory, file_name)
    spectrum_instance = Spectrum(file_path)
    spectrum_list.append(spectrum_instance)
print(len(spectrum_list))

#creates new array with only spectrums with less than 740 counts in the 3550 kev  and above range corresponds to channel 6375
new_spectrum_list = []

for spectrum_instance in spectrum_list:
    counts_above_3550 = sum(spectrum_instance.channel[6375:])
    if counts_above_3550 <= 740:
        new_spectrum_list.append(spectrum_instance)
        
spectrum_list = new_spectrum_list
print(len(spectrum_list))


bg_gaussian = [0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018956, 0.00018956, 0.00018956, 0.00018956,
       0.00018956, 0.00018957, 0.00018957, 0.00018957, 0.00018958,
       0.0001896 , 0.00018965, 0.00018974, 0.00018993, 0.00019029,
       0.00019098, 0.00019225, 0.00019446, 0.00019823, 0.00020441,
       0.00021419, 0.0002291 , 0.00025103, 0.00028206, 0.00032433,
       0.00037967, 0.00044918, 0.00053281, 0.00062893, 0.00073405,
       0.00084283, 0.00094836, 0.00104288, 0.0011186 , 0.00116881,
       0.00118886, 0.00117684, 0.00113389, 0.00106403, 0.00097355,
       0.00087004, 0.00076138, 0.00065476, 0.00055597, 0.00046897,
       0.00039583, 0.00033699, 0.00029158, 0.00025791, 0.00023389,
       0.0002174 , 0.00020648, 0.00019952, 0.00019524, 0.00019269,
       0.00019124, 0.00019043, 0.00019   , 0.00018977, 0.00018966]
#fits gaussian and shifts channel

# Define the Gaussian function
def gaussian(x, amplitude, mean, stddev, c):
    return amplitude * np.exp(-((x - mean) / stddev) ** 2) + c

def cost_function(params, x, y):
    return np.sum((gaussian(x, *params) - y) ** 2)

#channel detremined from energy line interested in, start and end of range determined by eye to ensure smooth fitting
spectral_range_start = 4670
spectral_line_channel = 4705
spectral_range_end = 4740
spectral_range_line = spectral_line_channel - spectral_range_start

epsilon = 0.013140455 # efficiency of the detector for the gamma ray energy
P = 1.0 # abundance of the gamma ray in the source material branching ratio

def redshift_spectrum(spectrum):
    # define the range of data to fit the gaussian to
    x_data = np.arange(spectral_range_start, spectral_range_end)
    y_data = spectrum.channel[x_data] 
    time = spectrum.livetime
    print(y_data)
    
    max_index = np.argmax(y_data)
    print(max_index)
    
    
    # initial guesses for the parameters of the gaussian function
    amplitude_guess = np.max(y_data)
    mean_guess = np.argmax(y_data)
    stddev_guess = 5.0
    c = 1.0

    initial_guess = [amplitude_guess, mean_guess, stddev_guess, c]

    # Minimize the cost function
    result = opt.minimize(cost_function, initial_guess, args=(np.arange(len(y_data)), y_data))

    # Get the optimized parameters
    params = result.x
    
     # shift the spectrum to center the peak on channel 1460 (channel 2635), and rescale for time
    center_of_mass = np.sum(y_data * np.arange(len(y_data))) / np.sum(y_data)
    com_adjustment = center_of_mass - max_index
    print(center_of_mass)
    shift = spectral_range_line - params[1] - com_adjustment
    print(shift)
    shifted_spectrum = np.roll(y_data, int(shift))
    
    #subtract background from shifted spectrum and clips any negative values to zero
    print(shifted_spectrum)
    

    # calculate the FWHM
    fwhm = 2.3548 * np.abs(params[2])
    
    print("std:", params[2])
    # calculate the area under the curve using FWHM
    area = np.sum(shifted_spectrum[int(np.argmax(shifted_spectrum) - fwhm) :int(np.argmax(shifted_spectrum) + fwhm + 1)])
    
    # Calculate the RSS and estimate the standard deviation of the errors
    y_fit = gaussian(np.arange(len(shifted_spectrum)), *params)
    rss = np.sum((shifted_spectrum - y_fit)**2)
    stddev_error = np.sqrt(rss / (len(shifted_spectrum) - len(params)))
    
    # Estimate the uncertainty in the area using error propagation
    dA_dh = np.sqrt(2 * np.pi) * fwhm / (2 * params[0])
    dA_dm = np.sqrt(2 * np.pi) * fwhm / (2.3548 * params[2]**2)
    print("da_dm", dA_dm)
    dA_ds = np.sqrt(2 * np.pi) * (params[1] - np.argmax(shifted_spectrum)) * fwhm / (2.3548 * params[2]**3)
    print("da_ds", dA_ds)
    area_error = np.sqrt( stddev_error**2 + dA_dm**2 * np.argmax(shifted_spectrum) + dA_ds**2 * params[2]**2)
    print("stderror", stddev_error)
    print("params2", params[2])
    
    
    

    return shifted_spectrum, params, area, area_error, time

# apply redshift_spectrum function to each spectrum in spectrum_list
shifted_spectrum_list = [redshift_spectrum(spectrum) for spectrum in spectrum_list]





activity_errors = []
sample_params = []
sample_activities = []
sample_durations = []
sample_spectra = []


for spectrum in shifted_spectrum_list:
    shifted_spectrum, params, area, area_error, time = spectrum
    sample_durations.append(time)
    sample_spectra.append(shifted_spectrum)
    activity = area / (epsilon  * P)
    sample_activities.append(activity)
    activity_errors.append(area_error)
    
    print("Spectrum around peak: ", shifted_spectrum)
    print("Amplitude:", params[0])
    print("Std Dev:", params[2])
    print("Area: ", area)
    print("Uncertainty:", area_error)
    print("Time:", time)
    
total_sample_duration = np.sum(sample_durations)
total_sample_spectrum = np.sum(sample_spectra, axis=0)

subtracted_spectrum = (total_sample_spectrum / total_sample_duration) - bg_gaussian
subtracted_spectrum = np.clip(subtracted_spectrum, a_min=0, a_max=None)

p0 = [np.max(subtracted_spectrum), np.argmax(subtracted_spectrum), 5.0, 1.0]
params, _ = curve_fit(gaussian, np.arange(len(subtracted_spectrum)), subtracted_spectrum, p0=p0)

# Generate the Gaussian array for the background
sample_gaussian = gaussian(np.arange(len(subtracted_spectrum)), *params) 
    
fwhm = 2.3548 * np.abs(params[2])
    
print("std:", params[2])
    # calculate the area under the curve using FWHM
# Define the start and end indices for the sum
start = int(np.argmax(subtracted_spectrum) - fwhm)
end = int(np.argmax(subtracted_spectrum) + fwhm + 1)

# Clip the indices to the valid range of the array
start = np.clip(start, 0, len(subtracted_spectrum))
end = np.clip(end, 0, len(subtracted_spectrum))

# Compute the area using the clipped indices
area = np.sum(subtracted_spectrum[start:end])
activity = area / (epsilon * P)
counts = (area * total_sample_duration) / epsilon    

    # Calculate the RSS and estimate the standard deviation of the errors
y_fit = gaussian(np.arange(len(subtracted_spectrum)), *params)
rss = np.sum((subtracted_spectrum - y_fit)**2)
stddev_error = np.sqrt(rss / (len(subtracted_spectrum) - len(params)))
    
    # Estimate the uncertainty in the area using error propagation
dA_dh = np.sqrt(2 * np.pi) * fwhm / (2 * params[0])
dA_dm = np.sqrt(2 * np.pi) * fwhm / (2.3548 * params[2]**2)
dA_ds = np.sqrt(2 * np.pi) * (params[1] - np.argmax(subtracted_spectrum)) * fwhm / (2.3548 * params[2]**3)
dA_dc = fwhm / (2.3548 * np.sqrt(area))
area_error = np.sqrt(dA_dh**2 * stddev_error**2 + dA_dm**2 * np.argmax(subtracted_spectrum) + dA_ds**2 * params[2]**2 + dA_dc**2)



          New instance of class created
filename:/Users/mohammadmaster/Desktop/2022-12-27_Montserrat_sample_32/Sample_32_A_137.Chn
The header is contained within the first 32 bytes: b'\xff\xff\x01\x00\x01\x0057 \xbf\x02\x001\xbe\x02\x0002Jan2310634\x00\x00\x00@'
Measurement date/time: 06:34:57 02-Jan-23
realtime=3600.0
livetime=3595.2200000000003
channel offset=0
number of channels=16384
one hundred and two: b'\x9a\xff'
One hundred and two (little endian): -102
Calibration constant A= 0
Calibration constant B= 0
Calibration constant C= 0
Detector description:Trans-SPEC DX100-P 13310315
Sample description:Montserrat Sample 32 - 1 hour series
          New instance of class created
filename:/Users/mohammadmaster/Desktop/2022-12-27_Montserrat_sample_32/Sample_32_A_123.Chn
The header is contained within the first 32 bytes: b'\xff\xff\x01\x00\x01\x0015 \xbf\x02\x002\xbe\x02\x0001Jan2311634\x00\x00\x00@'
Measurement date/time: 16:34:15 01-Jan-23
realtime=3600.0
livetime=3595.2400000000002
ch

In [345]:
print(counts )

282018.2812780989


In [346]:
print(area_error)

185.34141249995167


In [347]:
print(activity)

0.4194587965086794


In [252]:
print(subtracted_spectrum)

[7.51645378e-06 7.61685604e-05 3.50611512e-05 2.18698558e-05
 5.36771440e-05 2.41116778e-05 4.21501191e-05 1.76265961e-05
 1.73591869e-05 4.53119199e-05]


In [188]:
np.sum(total_sample_spectrum)

87284

In [339]:
total_sample_duration

284033.92

In [254]:
fwhm

0

In [239]:
dA_dh

0.0

In [237]:
dA_dm

0.0

In [238]:
stddev_error

2.5094484854559883e-05

In [255]:
params

array([-3.92865393e-06,  1.72989503e+00, -5.85300591e-03,  3.40852665e-05])

In [241]:
dA_dc

0.0

In [340]:
params[0] * total_sample_duration

58.29421149336165

In [183]:
print(area)

0.0
